# Lab 5: Naive Bayes and Random Forests
In this lab we'll take a closer look the [Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) and [Random Forests](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) classifiers.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
%matplotlib inline

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Naive Bayes
Naive Bayes is famously effective for spam email classification. We'll try our hand at applying NB on the [Spam email dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) from the UCI data repository. Download the dataset and move it to your lab data directory.

In [2]:
fname = 'data/SMSSpamCollection'
df = pd.read_table(fname, names=['class', 'text'])
df.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Multinomial Naive Bayes
Given collection of documents $d^{(1)}, d^{(2)}, ..., d^{(N)}$ with corresponding labels $y^{(i)}$, we first need to extract features from each document. We'll do this by computing word counts of each word in each document. This is commonly referred to as a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) representation.

After doing some preprocessing of our data, we'll have: $\{(x^{(i)}, y^{(i)})\}_{i=1}^N$, where:
* $x^{(i)} \in R^s$ is a word count feature vector for document $i$, and $s$ is the size of the vocabulary. For the purposes of this lab, we define our vocabulary to be the set of words that appear in our dataset of documents.
$$ x^{(i)} = (\theta^{(i)}_1, \theta^{(i)}_2, ..., \theta^{(i)}_s)$$
$$\theta^{(i)}_k = \text{ the number of times word $k$ appears in document $i$}$$


* $y^{(i)} \in \{0, 1\}$ is its class. In the spam email classification task, 0 denotes spam, 1 denotes ham.

Before we can use sklearn's Naive Bayes classifier, we need to compute word count feature vectors. We can do this manually like so:

In [3]:
import string

def get_all_words(docs):
    '''
    docs: pandas.Series of strings
    Returns: set of words(strings)
    '''
    # Compute all the words in the docs
    words = set()
    for d in docs:
        # this will strip punctuation
        # ref: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        d = d.translate(dict.fromkeys(string.punctuation))
        words = words.union(set(d.split()))
    return words

def make_word_cnt_feats(docs, vocabulary=None):
    '''
    docs: iterable(ie: list or pandas.Series) of strings
    vocabulary: set of words(strings) that appear in docs
    Returns: numpy matrix of the word count features
    '''
    if vocabulary is None:
        vocabulary = get_all_words(docs)
    words = sorted(vocabulary)
    word_dict = {w: idx for idx, w in enumerate(words)}
    
    # feature matrix will be of size n x (size of vocabulary + 1)
    # add 1 to account for words that dont appear in the vocabulary to avoid random issues
    vocab_size = len(words)
    word_cnt_features = np.zeros((len(docs), len(words) + 1))
    
    for idx in range(len(docs)):
        doc = docs[idx]
        doc = doc.translate(dict.fromkeys(string.punctuation))
        words = doc.split()
        for w in words:
            word_idx = word_dict.get(w, vocab_size)
            word_cnt_features[idx, word_idx] += 1
            
    return word_cnt_features

In [4]:
vocabulary = get_all_words(df['text'])
feature_matrix = make_word_cnt_feats(df['text'], vocabulary)
print('Feature matrix shape: {}'.format(feature_matrix.shape))

Feature matrix shape: (5572, 15692)


It turns out that sklearn actually has a built-in method to do this kind of word count featurization! (But of course, it's good to get your hands dirty with manual featurization from time to time to understand all the ingredients of these algorithms). See sklearn's [sklearn.feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) function for more details.                                                                 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
word_cnt_feats = count_vectorizer.fit_transform(df['text'])
word_cnt_feats # this is a sparse matrix
# looks like this gives us fewer columns, which means we didn't prune
# a bunch of unwanted terms(maybe words with numbers?) in our code above

<5572x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) has a bunch of useful functions that you should checkout. The important ones that we'll use are:
* [fit](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit): this learns the vocabulary of the input collection of strings/"documents"
* [transform](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform): given a collection of documents, this produces the word count features for the documents
* fit_transform: calls fit, then transform

Note that the word count feature matrix(aka document-term matrix) is in sparse format. This is not an issue for the sklearn models. But if you want to expand them to non-sparse numpy arrays, use the toarray function. Note that this can be problematic for bigger datasets! We can somewhat get around this b/c this dataset is small enough that we can fully expand a 5k by 8k matrix(since it's sparse enough).

In [6]:
X = word_cnt_feats.toarray()
Y = (df['class'] == 'ham')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)

Now that we have the data in the document-term matrix format, we can use the Naive Bayes classifier!

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
nb = MultinomialNB(alpha=1)
nb.fit(X_train, Y_train)
preds = nb.predict(X_test)

In [8]:
print('Accuracy: {:.3f}'.format(accuracy_score(preds, Y_test)))
print('F1 Score: {:.3f}'.format(f1_score(preds, Y_test)))
print('ROC AUC Score: {:.3f}'.format(roc_auc_score(preds, Y_test)))

Accuracy: 0.975
F1 Score: 0.985
ROC AUC Score: 0.943


## Naive Bayes Tasks
Now it's your turn to use it on the Naive Bayes model. Pick an evaluation metric to use for cross validation like precision, recall, F1 score, ROC AUC score, etc. In the spam classification setting, do you think it makes sense to optimize for higher precision? recall? f1 score?

0) Pick an evaluation metric.

1) Note that the code above fixed alpha to 1. But as we've learned in class and last week's lab, we should use cross validation for any kind of parameter search. Use [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) to do the cross validation over 5 folds to find the alpha that works best on the training set.

In [9]:
import pdb

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import KFold

def cv_multinom_bayes(x_train, y_train, k_folds_num, alpha_list):
    y_train_np = y_train.values
    kf = KFold(n_splits = k_folds_num)
    # dictionary of results:
    results_nb = {}
    # split data into k-folds, and loop over each fold
    for train_idx, test_idx in kf.split(x_train):
        # split into k-folds
        x_split_train, x_split_test = x_train[train_idx], x_train[test_idx]
        y_split_train, y_split_test = y_train_np[train_idx], y_train_np[test_idx]
        #loop over different alpha values
        for alpha_value in alpha_list:
            # intialize classifier and fit it to the split training set
            nb_clf = MultinomialNB(alpha = alpha_value)
            try:
                nb_clf.fit(x_split_train, y_split_train)
            except:
                pdb.set_trace()
                
            # make predictions
            y_pred = nb_clf.predict(x_split_test)
            # intialize the main dictionary key
            model_key = (alpha_value)
            results_nb[model_key] = {}
            # write evaluation results to dictionary
            results_nb[model_key]['Accuracy']= results_nb[model_key].get('Accuracy', 0) + accuracy_score(y_pred, y_split_test)/k_folds_num
            results_nb[model_key]['Precision']= results_nb[model_key].get('Precision', 0) + precision_score(y_pred, y_split_test)/k_folds_num
            results_nb[model_key]['Recall']= results_nb[model_key].get('Recall', 0) + recall_score(y_pred, y_split_test)/k_folds_num
            results_nb[model_key]['F1']= results_nb[model_key].get('F1', 0) + f1_score(y_pred, y_split_test)/k_folds_num
   #print results
    for model, model_perf in results_nb.items():
        print("Model parameters: {}".format(model, model_perf))
        for eva_metric, eva_value in model_perf.items():
            print("{}: {}".format(eva_metric, eva_value))
        print("————————————————————————————————")
    #return the results, in case I want to do anything with it.
    return results_nb

In [16]:
alphas = [1, 5, 10, 25, 100] # you should vary these 

cv_multinom_bayes(X_train, Y_train, 5, alpha_list= alphas)


Model parameters: 1
Accuracy: 0.1970059880239521
Precision: 0.19825783972125435
Recall: 0.19825783972125435
F1: 0.19825783972125435
————————————————————————————————
Model parameters: 5
Accuracy: 0.19221556886227545
Precision: 0.2
Recall: 0.19133333333333333
F1: 0.19557069846678027
————————————————————————————————
Model parameters: 10
Accuracy: 0.18802395209580838
Precision: 0.2
Recall: 0.18697068403908795
F1: 0.19326599326599325
————————————————————————————————
Model parameters: 25
Accuracy: 0.181437125748503
Precision: 0.2
Recall: 0.18050314465408807
F1: 0.1897520661157025
————————————————————————————————
Model parameters: 100
Accuracy: 0.17305389221556886
Precision: 0.2
Recall: 0.17289156626506025
F1: 0.18546042003231017
————————————————————————————————


{1: {'Accuracy': 0.1970059880239521,
  'F1': 0.19825783972125435,
  'Precision': 0.19825783972125435,
  'Recall': 0.19825783972125435},
 5: {'Accuracy': 0.19221556886227545,
  'F1': 0.19557069846678027,
  'Precision': 0.2,
  'Recall': 0.19133333333333333},
 10: {'Accuracy': 0.18802395209580838,
  'F1': 0.19326599326599325,
  'Precision': 0.2,
  'Recall': 0.18697068403908795},
 25: {'Accuracy': 0.181437125748503,
  'F1': 0.1897520661157025,
  'Precision': 0.2,
  'Recall': 0.18050314465408807},
 100: {'Accuracy': 0.17305389221556886,
  'F1': 0.18546042003231017,
  'Precision': 0.2,
  'Recall': 0.17289156626506025}}

2) Now that you've found the optimal alpha using cross validation over the training set, train a Naive Bayes model using the optimal alpha over the entirety of the training data and evaluate this model on the test set.

In [41]:
# CODE

Further Questions:
* Naive Bayes has a seemingly stringent independence assumption. Do you think the variables in this email classification problem(word count vectors) are really independent? Does this matter? Why/why not?

## Random Forests
Two weeks ago, we played around with decision trees. In general, practictioners almost never use decision trees over Random Forests. Recall from lecture, that Random Forests takes some number of subsamples of the data:
$S_1, S_2, ..., S_k \subseteq \{(x^{(i)}, y^{(i)}\}_{i=0}^N$. On each subsample of the data, $S_i$, the Random Forests algorithm trains a decision tree on $S_i$.

At classification time, when we get a datapoint $x$, the Random Forests classifier runs $x$ through decision trees $D_1, D_2, ..., D_k$, which produces a "probability" estimate of each class:

$$P(y = 1 | x) = \frac{\text{number of decision trees that classified $x$ as 1}}{k}$$

$$P(y = 0 | x) = \frac{\text{number of decision trees that classified $x$ as 1}}{k}$$

Then from there, we get a class prediction by picking $y = 1$ if $P(y = 1 | x) > T$, for some threshold probability $T$.


Instead of using the spam dataset to try out Random Forests, we'll re-use the credit loan data from Assignment 2.

In [5]:
df = pd.read_csv("credit-data.csv")

Do whatever data cleaning, data imputing, etc you need to do.

In [6]:
def show_nulls(df):
	return df.isna().sum().sort_values(ascending=False)

def fill_whole_df_with_mean(df):
    num_cols = len(df.columns)
    for i in range(0, num_cols):
        df.iloc[:,i] = fill_col_with_mean(df.iloc[:,i])
    return

def fill_col_with_mean(df):
	return df.fillna(df.mean())

In [7]:
from sklearn.model_selection import train_test_split
features = ['MonthlyIncome', 'DebtRatio', 'age', 'NumberOfTimes90DaysLate','NumberOfOpenCreditLinesAndLoans'] # Pick the features you want
df_features = df[features]
df_target = df['SeriousDlqin2yrs']
X_train, X_test, Y_train, Y_test = train_test_split(df_features, df_target, test_size=0.2)

In [8]:
fill_whole_df_with_mean(X_train)
fill_whole_df_with_mean(X_test)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
from sklearn.preprocessing import scale
def scale_a_df(df, features_list):
    temp_scaled = scale(df[features_list])
    #return a DF
    return pd.DataFrame(temp_scaled, columns= df.columns)

In [10]:
X_train = scale_a_df(X_train, features)

In [11]:
X_test = scale_a_df(X_test, features)

Split the data into train/test sets.

In [46]:
# split data

## Random Forest Parameters

The Random Forest parameters are more or less the same as those for Decision Trees. The only additional parameter is n_estimators, which is the number of decision trees to grow for the algorithm.
* n_estimators : integer, the number of trees in the forest.
* criterion : string,  “gini” or “entropy” 
* max_features : int, float, string or None, the number of features to consider when looking for the best split
* max_depth : integer, the maximum depth of the tree
* min_samples_split : int, float, the minimum number of samples required to split an internal node:
* min_samples_leaf : int, float, optional (default=1), the minimum number of samples required to be at a leaf node:
* min_weight_fraction_leaf : float, optional 
* max_leaf_nodes : int or None, optional (default=None)

0) What do you think will be the effect of increasing the n\_estimators parameter? If n_estimators = 1, is this any different to using a decision tree classifier(assuming you set all the other parameters like min_sample_split, max_depth, etc the same way in both cases)?

1) Pick an evaluation metric.

2) Pick a few values for each of the random forest parameters for cross validation.

In [12]:
# max_features: the max number of features each decision tree is allowed to try.
# increasing max_features generally improves performance, but decreases the speed of the algorithm
max_feats = ["auto", "sqrt", "log2", 0.2]

# n_estimators: the number of decision trees you're going to build.
# higher the better, but of course the higher it is the slower the model is going to be.
num_dtrees = [5, 10, 20]

# max_depth: maximum depth of decision trees
depth_list = [3,5,8]

# criterion: gini or entropy
criterion_list = ["gini", "entropy"]

# min_samples_split: the minimum number of samples required to split an internal node
min_split_list = [2,3,5]

# min_samples_leaf: the minimum number of samples required to be at a leaf node:
min_leaf_list = [1,2,3]


In last weeks lab, we saw how to grid search over all possible model parameters and cross validation folds. To refresh your memory, below is a slightly updated version of that code.

In [ ]:
# Example of a manual grid search for LogisticRegression on random data
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
def sample_manual_grid_search(n_samples, n_feats, splits):
    # make the fake data
    X = np.random.randint(0, 10, (n_samples, n_feats))
    Y = (np.random.random(n_samples) > 0.5) # random true/false
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)
    
    # params to search over
    penalties = ['l1', 'l2']
    c_vals = [10**i for i in range(-2, 3)]

    kf = KFold(n_splits=splits)
    rows = [] # store dicts of the cross validation results for each fold
    for p in penalties:
        for c in c_vals:

            fold_dict = {
                    # NOTE: these key values must be exactly as they're spelled in
                    # the sklearn model!
                    'penalty': p,
                    'C': c,
            }
            fold_results = {}
            for fold_num, (train_idx, test_idx) in enumerate(kf.split(X_train, y_train)):
                x_split_train, x_split_test = X_train[train_idx], X_train[test_idx]
                y_split_train, y_split_test = y_train[train_idx], y_train[test_idx]
                    
                model = LogisticRegression(**fold_dict)
                model.fit(x_split_train, y_split_train)
                pred_probs = model.predict_proba(x_split_test)
                loss = roc_auc_score(y_split_test, pred_probs[:, 0])
                
                # Keep track of the performance of the model on each fold and store the result
                fold_results['fold_{}'.format(fold_num+1)] = loss
                    
            # now that we have all the fold results, update the dict containing the model params
            # with the performance results
            fold_dict.update(fold_results)
            fold_dict['avg_score'] = np.mean(fold_results.values())
            rows.append(fold_dict)

    # we can create a dataframe from a list of dicts. This produces a dataframe whose columns are the keys of
    # the dicts in the list. Recall that rows is a list of dicts
    res_df = pd.DataFrame(rows)
    return res_df


In [61]:
cv_results = sample_manual_grid_search(n_samples=100, n_feats=20, splits=5)
print("Grid Search Results for sample data with Logistic Regression:")
cv_results.head()

Grid Search Results for sample data with Logistic Regression:


,C,avg_score,fold_1,fold_2,fold_3,fold_4,fold_5,penalty
0,0.01,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,l1
1,0.10,0.635608,0.314286,0.740741,0.694444,0.657143,0.771429,l1
2,1.00,0.633016,0.514286,0.666667,0.555556,0.685714,0.742857,l1
3,10.00,0.606931,0.600000,0.592593,0.527778,0.542857,0.771429,l1
4,100.00,0.550582,0.571429,0.481481,0.500000,0.542857,0.657143,l1


3) Do the same kind of grid search cross validation for Random Forests classifier. Which set of parameters gets the best average score over all folds?

In [42]:
def custom_predictions(clf, threshold = 0.7, x_test = X_test):
    # threshold = the probability threshold for something to be a 0.
    # generate array with predicted probabilities
    pred_array = clf.predict_proba(x_test)

    # initialize an empty array for the predictions
    pred_generated = np.array([])

    # predict the first entry
    if pred_array[0][0] >= threshold:
        pred_generated = np.hstack([pred_generated, 0])
    else:
        pred_generated = np.hstack([pred_generated, 1])

    # loops over the rest of the array
    for i in range(1,len(x_test)):
        if pred_array[i][0] >= threshold:
            pred_generated = np.vstack([pred_generated, 0])
        else:
            pred_generated = np.vstack([pred_generated, 1])

    # return an np.array
    return  pred_generated



from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import KFold


def grid_cv_RF(x_train, y_train, max_feats, num_dtrees_list, depth_list, criterion_list, k_folds_num):
    kf = KFold(n_splits = k_folds_num)
    # dictionary of results:
    results_rf = {}
    # split data into k-folds, and loop over each fold
    for train_idx, test_idx in kf.split(x_train):
        # split into k-folds
        x_split_train, x_split_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
        y_split_train, y_split_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
        # loop over max_features
        for feature_number in max_feats:
            # loop over n_estimators
            for num_trees in num_dtrees_list:
                #loop over max_depth
                for depth_num in depth_list:
                    #loop over information gain criterion
                    for criterion_choice in criterion_list:
                        rf_clf = RandomForestClassifier(max_features = feature_number, n_estimators = num_trees, max_depth = depth_num, criterion = criterion_choice)
                        rf_clf.fit(x_split_train, y_split_train)
                        # generate prediction using threshold of 0.3 for labels 0
                        y_pred = custom_predictions(rf_clf, threshold = 0.7, x_test = x_split_test)
                        # intialize the main dictionary key
                        model_key = (feature_number, num_trees, depth_num, criterion_choice)
                        # write resuls to dictionary
                        results_rf[model_key] = {}
                        # write evaluation results to dictionary
                        results_rf[model_key]['Accuracy']= results_rf[model_key].get('Accuracy', 0) + accuracy_score(y_pred, y_split_test)/k_folds_num
                        results_rf[model_key]['Precision']= results_rf[model_key].get('Precision', 0) + precision_score(y_pred, y_split_test)/k_folds_num
                        results_rf[model_key]['Recall']= results_rf[model_key].get('Recall', 0) + recall_score(y_pred, y_split_test)/k_folds_num
                        results_rf[model_key]['F1']= results_rf[model_key].get('F1', 0) + f1_score(y_pred, y_split_test)/k_folds_num
    #print results
    for model, model_perf in results_rf.items():
        print("Model parameters: {}".format(model, model_perf))
        for eva_metric, eva_value in model_perf.items():
            print("{}: {}".format(eva_metric, eva_value))
        print("————————————————————————————————")
    #return the results, in case I want to do anything with it.
    return results_rf

In [43]:
X_train.head()

,MonthlyIncome,DebtRatio,age,NumberOfTimes90DaysLate,NumberOfOpenCreditLinesAndLoans
0,-2.171613e-01,-0.248654,-1.536492,-0.079579,-0.842833
1,1.380547e-16,10.752528,-0.182472,-0.079579,2.028727
2,-3.168133e-01,-0.248409,2.254764,-0.079579,0.305791
3,1.665707e-01,-0.248523,0.765342,-0.079579,0.114354
4,1.380547e-16,2.656153,0.426837,-0.079579,0.497228


In [44]:
# important random forest parameters:

# max_features: the max number of features each decision tree is allowed to try.
# increasing max_features generally improves performance, but decreases the speed of the algorithm
max_feats = ["auto", "sqrt", "log2", 0.2]

# n_estimators: the number of decision trees you're going to build.
# higher the better, but of course the higher it is the slower the model is going to be.
num_dtrees_list = [5, 10, 20]

# max_depth: maximum depth of decision trees
depth_list = [3,5,8]

# criterion: gini or entropy
criterion_list = ["gini", "entropy"]

# min_samples_split: the minimum number of samples required to split an internal node
min_split_list = [2,3,5]

# min_samples_leaf: the minimum number of samples required to be at a leaf node:
min_leaf_list = [1,2,3]

In [45]:
resultsDF = grid_cv_RF(X_train, Y_train, max_feats, num_dtrees_list, depth_list, criterion_list, 5)

Model parameters: ('auto', 5, 3, 'gini')
Accuracy: 0.17202072538860103
Precision: 0.06525345622119816
Recall: 0.1308687615526802
F1: 0.08708487084870849
————————————————————————————————
Model parameters: ('auto', 5, 3, 'entropy')
Accuracy: 0.17199024687595246
Precision: 0.06488479262672811
Recall: 0.1308550185873606
F1: 0.08675292667898953
————————————————————————————————
Model parameters: ('auto', 5, 5, 'gini')
Accuracy: 0.17177689728741236
Precision: 0.0656221198156682
Recall: 0.12875226039783
F1: 0.08693528693528693
————————————————————————————————
Model parameters: ('auto', 5, 5, 'entropy')
Accuracy: 0.17183785431270954
Precision: 0.06617511520737326
Recall: 0.12890484739676838
F1: 0.0874543239951279
————————————————————————————————
Model parameters: ('auto', 5, 8, 'gini')
Accuracy: 0.1713806766229808
Precision: 0.07023041474654378
Recall: 0.12370129870129871
F1: 0.08959435626102294
————————————————————————————————
Model parameters: ('auto', 5, 8, 'entropy')
Accuracy: 0.17141115513

Recall: 0.13381642512077294
F1: 0.07391594396264176
————————————————————————————————
Model parameters: (0.2, 10, 5, 'gini')
Accuracy: 0.17195976836330387
Precision: 0.06506912442396313
Recall: 0.13049907578558226
F1: 0.08683886838868389
————————————————————————————————
Model parameters: (0.2, 10, 5, 'entropy')
Accuracy: 0.17202072538860103
Precision: 0.06525345622119816
Recall: 0.1308687615526802
F1: 0.08708487084870849
————————————————————————————————
Model parameters: (0.2, 10, 8, 'gini')
Accuracy: 0.1710149344711978
Precision: 0.0687557603686636
Recall: 0.12189542483660132
F1: 0.08791985857395404
————————————————————————————————
Model parameters: (0.2, 10, 8, 'entropy')
Accuracy: 0.1715635476988723
Precision: 0.06728110599078341
Recall: 0.12629757785467127
F1: 0.08779314491882141
————————————————————————————————
Model parameters: (0.2, 20, 3, 'gini')
Accuracy: 0.17202072538860103
Precision: 0.06525345622119816
Recall: 0.1308687615526802
F1: 0.08708487084870849
——————————————————————

In [50]:
results = pd.DataFrame.from_dict(resultsDF).transpose()

In [59]:
results.sort_values("F1",ascending=False).head(5)

,Accuracy,F1,Precision,Recall
"(log2, 10, 8, entropy)",0.171320,0.089877,0.070783,0.123077
"(log2, 10, 8, gini)",0.171289,0.089825,0.070783,0.122880
"(auto, 5, 8, gini)",0.171381,0.089594,0.070230,0.123701
"(sqrt, 5, 8, entropy)",0.171015,0.089225,0.070599,0.121203
"(sqrt, 20, 8, gini)",0.171350,0.089151,0.069677,0.123732


The best model uses log2, 10, 8, entropy.

## Grid Search Cross Validation
Writing 8x nested for-loops to search over all parameters can be a bit cumbersome. Luckily, sklearn has a very nice [GridSearchCV class](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) that can do all this grid searching and cross validating for us!

GridSearchCV requires:
* an estimator object(IE: LogisticRegression(), SVC(), KNeighborsClassifier(), etc)
* param_grid: a dictionary mapping parameter names to lists of values to search over. We'll see an example below
* scoring: a string. See the valid inputs here: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
* n_jobs: number of jobs to run in parallel to speed up the computation. GridSearchCV can take a long time if you have a lot of different parameter combinations to try out. So it's important to set this parameter if your laptop/desktop has a lot of cores that you want to take advantage of.
* cv: int, number of cross validation folds to use
* refit: boolean, if true, at the end of the Grid Search, GridSearchCV will refit the entire data with a model using the optimal parameters(you can then access this with the GridSearchCV.best\_parameters_ attribute).

Here's an example of how we'd use it for some random data:

In [2]:
# Example of how to use GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

def sample_cross_val_param_search():
    # parameters to search over
    penalties = ['l1', 'l2']
    c_vals = [10**i for i in range(-2, 3)]
    param_grid = {'penalty': penalties, 'C': c_vals}
    
    # make some fake data
    n_samples = 1000
    feats = 10
    x_train = np.random.random((n_samples, feats))
    y_train = (np.random.random(n_samples) > 0.5) # randomly generate True/False labels
    
    grid = GridSearchCV(LogisticRegression(), param_grid, 'f1', cv=5)
    grid.fit(x_train, y_train)
    return grid
grid = sample_cross_val_param_search()
results = pd.DataFrame(grid.cv_results_)
print("GridSearchCV results:")
results.head(3)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages

GridSearchCV results:


/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/li

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_penalty,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.002169,0.001586,0.000000,0.000000,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",9,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.001739,0.000721,0.000000,0.000000
1,0.070426,0.000892,0.004167,0.002097,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",8,0.0,0.0,...,0.0,0.005236,0.0,0.0,0.020833,0.0,0.138558,0.000212,0.008333,0.002568
2,0.001160,0.000819,0.000000,0.000000,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",9,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000245,0.000008,0.000000,0.000000


In [4]:
grid.best_score_

0.43517228909366606

In [15]:
print('Best estimator: {}'.format(grid.best_estimator_))
print('Best Parameters: {}'.format(grid.best_params_))

Best estimator: LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Best Parameters: {'penalty': 'l1', 'C': 0.1}


4) Use GridSearchCV to do the same parameter search you did above. What is the best performing set of parameters? Do they match up with what you had above?

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import KFold

In [19]:
# important random forest parameters:

# max_features: the max number of features each decision tree is allowed to try.
# increasing max_features generally improves performance, but decreases the speed of the algorithm
max_feats = ["auto", "sqrt", "log2", 0.2]

# n_estimators: the number of decision trees you're going to build.
# higher the better, but of course the higher it is the slower the model is going to be.
num_dtrees_list = [5, 10, 20]

# max_depth: maximum depth of decision trees
depth_list = [3,5,8]

# criterion: gini or entropy
criterion_list = ["gini", "entropy"]

# min_samples_split: the minimum number of samples required to split an internal node
min_split_list = [2,3,5]

# min_samples_leaf: the minimum number of samples required to be at a leaf node:
min_leaf_list = [1,2,3]

In [25]:
param_grid = {'max_features': max_feats, 'n_estimators' : num_dtrees_list, "max_depth": depth_list, "criterion": criterion_list}


In [27]:
grid = GridSearchCV(RandomForestClassifier(), param_grid, 'f1', cv=5)
grid.fit(X_train, Y_train)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': ['auto', 'sqrt', 'log2', 0.2], 'n_estimators': [5, 10, 20], 'max_depth': [3, 5, 8], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [29]:
bestrf = grid.best_estimator_

In [30]:
bestrf.feature_importances_

array([0.02499886, 0.0453088 , 0.098854  , 0.79497538, 0.03586296])

In [42]:
pd.DataFrame(grid.cv_results_).head(5)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/li

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,rank_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.063152,0.009816,0.382086,0.388360,gini,3,auto,5,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",57,...,0.420849,0.425739,0.436503,0.450120,0.396201,0.417700,0.013113,0.008627,0.089498,0.091896
1,0.101559,0.007434,0.408619,0.410408,gini,3,auto,10,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",53,...,0.332374,0.349847,0.445278,0.460479,0.455233,0.458471,0.006029,0.000087,0.050924,0.051974
2,0.206530,0.012888,0.415069,0.416624,gini,3,auto,20,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",48,...,0.447220,0.462498,0.416063,0.431153,0.433485,0.439308,0.006607,0.001077,0.058010,0.058453
3,0.051430,0.006089,0.433975,0.433992,gini,3,sqrt,5,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",34,...,0.343262,0.365527,0.440332,0.448331,0.446144,0.449762,0.003621,0.002066,0.047011,0.034428
4,0.102455,0.007406,0.436541,0.438533,gini,3,sqrt,10,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",30,...,0.434894,0.449456,0.436061,0.449600,0.454716,0.460171,0.003220,0.000117,0.030789,0.029214


5) As mentioned earlier, GridSearchCV.best\_estimator\_ will contain a model with parameters set to the best performing set of parameters fitted on the entire training set. Use this best estimator on the test set.

In [37]:
bestrf = grid.best_estimator_
bestrf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1])

6) Random Forests has a [feature importances](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_) attribute. Which features are the most important?

In [39]:
X_train.columns

Index(['MonthlyIncome', 'DebtRatio', 'age', 'NumberOfTimes90DaysLate',
       'NumberOfOpenCreditLinesAndLoans'],
      dtype='object')

In [38]:
bestrf.feature_importances_

array([0.02499886, 0.0453088 , 0.098854  , 0.79497538, 0.03586296])

No. of times 90 days late seems to be the most important!

4) Rerun the classifier with a different scoring metric. Do the feature importances change? Which features are most important now?

In [43]:
scoring = {'accuracy': 'accuracy', 'precision': 'precision', "roc": "roc_auc"}

In [46]:
grid = GridSearchCV(RandomForestClassifier(), param_grid, scoring=scoring, cv=5, refit='roc')
grid.fit(X_train, Y_train)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': ['auto', 'sqrt', 'log2', 0.2], 'n_estimators': [5, 10, 20], 'max_depth': [3, 5, 8], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit='roc', return_train_score='warn',
       scoring={'accuracy': 'accuracy', 'precision': 'precision', 'roc': 'roc_auc'},
       verbose=0)

In [48]:
pd.DataFrame(grid.cv_results_)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_roc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/xiangyu/anaconda3/l

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_precision,mean_test_roc,mean_train_accuracy,mean_train_precision,mean_train_roc,param_criterion,param_max_depth,...,split4_train_precision,split4_train_roc,std_fit_time,std_score_time,std_test_accuracy,std_test_precision,std_test_roc,std_train_accuracy,std_train_precision,std_train_roc
0,0.059374,0.019935,0.867670,0.690167,0.755315,0.868089,0.694492,0.756140,gini,3,...,0.695152,0.756373,0.010633,0.009560,0.002399,0.015506,0.003926,0.000596,0.003223,0.003067
1,0.116034,0.023296,0.866939,0.693577,0.756573,0.867267,0.698188,0.762213,gini,3,...,0.688478,0.763181,0.014529,0.003572,0.002455,0.013795,0.005194,0.001649,0.012032,0.002802
2,0.199478,0.035406,0.866360,0.698181,0.760356,0.866787,0.703005,0.764094,gini,3,...,0.697316,0.765062,0.006416,0.000811,0.002955,0.015095,0.007091,0.001846,0.019597,0.000747
3,0.050827,0.013567,0.863708,0.695033,0.751129,0.864364,0.701152,0.756152,gini,3,...,0.713333,0.756912,0.003450,0.001130,0.006013,0.020515,0.005547,0.006125,0.010932,0.002344
4,0.113059,0.022083,0.861453,0.721771,0.758836,0.862124,0.719639,0.764660,gini,3,...,0.693060,0.765154,0.012871,0.001845,0.008608,0.039284,0.005770,0.007735,0.023698,0.002904
5,0.206890,0.036665,0.868432,0.693759,0.760162,0.867945,0.692374,0.764664,gini,3,...,0.697595,0.765335,0.015547,0.002631,0.002528,0.010155,0.004714,0.000638,0.009739,0.002372
6,0.052485,0.013002,0.864745,0.693688,0.755310,0.866276,0.701285,0.759811,gini,3,...,0.718922,0.763218,0.000859,0.000206,0.003261,0.015974,0.003977,0.001677,0.016134,0.004515
7,0.104311,0.021540,0.867579,0.693076,0.758407,0.868059,0.696486,0.762994,gini,3,...,0.702863,0.759579,0.005748,0.002081,0.002630,0.014115,0.004857,0.001000,0.007786,0.002418
8,0.232229,0.037121,0.865446,0.697589,0.760321,0.866985,0.705620,0.764559,gini,3,...,0.689144,0.762116,0.041846,0.005479,0.001629,0.024928,0.005573,0.001776,0.009866,0.001468
9,0.059784,0.020587,0.842405,0.604066,0.754855,0.843282,0.606285,0.760616,gini,3,...,0.000000,0.760741,0.018258,0.002036,0.005943,0.332516,0.006782,0.007022,0.312112,0.002211


## References
* https://etav.github.io/projects/spam_message_classifier_naive_bayes.html
* http://scikit-learn.org/stable/modules/naive_bayes.html
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html